In [ ]:
# calibrateCamera(.pickle) 
# Input: 
#     ES 143 AprilBoard datafile (AprilBoards.pickle)
# Returns:
#     calMatrix (K, callibration matrix) 
#     distCoeffs


#####################################################
# PROBABLY MOVE THIS IMPORTING CELL TO MAIN SCRIPT? #
#####################################################

# Install pupil_apriltags package. This a wrapper for the Apriltags3 library 
#   developed by AprilRobotics.
# Documentation at https://github.com/pupil-labs/apriltags

%pip install pupil-apriltags

# install opencv
!pip install opencv-python

# Import any required libraries here
import cv2                               # OpenCV
import numpy as np                       # numpy
import os
from pupil_apriltags import Detector
import requests
import pickle
import glob   # filename and path management for file I/O


# Use this line to make matplotlib plot inline (only need to call it once when importing matplotlib)
%matplotlib inline

import matplotlib.pyplot as plt

# unzip calib files and import 
# ! unzip -n ./resources.zip

# data = pickle.load(open('./AprilBoards.pickle', 'rb'))
# at_coarseboard = data['at_coarseboard']
# at_fineboard = data['at_fineboard']

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
# pickle_file is .pickle
# input_images is .zip
def calibrateCamera(pickle_file): 

    # helper function
    # Returns: 
    #   imgpoints -- Nx2 numpy array of (x,y) image coords
    #   objpoints -- Nx3 numpy array of (X,Y,Z=0) board coordinates (in inches)
    #   tag_ids -- Nx1 list of tag IDs
    def detect_aprilboard(img, board, apriltag_detector):

        imgpoints=[]
        objpoints=[]
        tagIDs=[]
        
        # detect april tags
        tags = apriltag_detector.detect(img, 
                                        estimate_tag_pose=False, 
                                        camera_params=None, 
                                        tag_size=None)
        
        if len(tags):
            # collect image coordinates of tag centers
            imgpoints = np.vstack([ sub.center for sub in tags ])

            # list of all detected tag_id's in image
            tagIDs = [ sub.tag_id for sub in tags ]

            # all board list-elements that contain one of the detected tag_ids
            objs=list(filter(lambda tagnum: tagnum['tag_id'] in tagIDs, board))    
        
            if len(objs):
                # gather the center coordinates from each of these board elements
                objpoints = np.vstack([ sub['center'] for sub in objs ])
        
        return imgpoints, objpoints, tagIDs

    # select all calibration images
    CALIBFILES = './resources_unzipped/calib0*.png'
    
    # choose pickle data for fine board
    data = pickle.load(open(pickle_file, 'rb'))
    at_coarseboard = data['at_coarseboard']
    at_fineboard = data['at_fineboard']
    BOARD = at_fineboard
   
    
    ###### BEGIN CALIBRATION SCRIPT

    # exit if no images are found or if BOARD is unrecognized
    images = glob.glob(CALIBFILES)
    assert images, "no calibration images matching: " + CALIBFILES
    assert BOARD==at_fineboard or BOARD==at_coarseboard, "Unrecognized AprilBoard"


    # initialize 3D object points and 2D image points
    calObjPoints = []
    calImgPoints = []

    # set up april tag detector (I use default parameters; seems to be OK)
    at_detector = Detector(families='tag36h11',
                        nthreads=1,
                        quad_decimate=1.0,
                        quad_sigma=0.0,
                        refine_edges=1,
                        decode_sharpening=0.25,
                        debug=0)


    for count,fname in enumerate(images):
        
        # read image and convert to grayscale if necessary
        orig = cv2.imread(fname)
        if len(orig.shape) == 3:
            img = cv2.cvtColor(orig, cv2.COLOR_RGB2GRAY)
        else:
            img = orig


        # detect apriltags and report number of detections
        imgpoints, objpoints, tagIDs = detect_aprilboard(img,BOARD,at_detector)
        
        # append detections if some are found
        if len(imgpoints) and len(objpoints):
                
            # append points detected in all images, (there is only one image now)
            calObjPoints.append(objpoints.astype('float32'))
            calImgPoints.append(imgpoints.astype('float32'))


    # convert to numpy array
    calObjPoints = np.array(calObjPoints)
    calImgPoints = np.array(calImgPoints)
        
    # calibrate the camera 
    reprojerr, calMatrix, distCoeffs, calRotations, calTranslations = cv2.calibrateCamera(
        calObjPoints, 
        calImgPoints, 
        img.shape,    # image H,W for initialization of the principal point
        None,         # no initial guess for the remaining entries of calMatrix
        None,         # initial guesses for distortion coefficients are all 0
        flags = None) # default contstraints (see documentation)

    # performance check (passed): 
    # print('RMSE of reprojected points:', reprojerr)
    return calMatrix, distCoeffs
    


In [ ]:
# test:
K, dist = calibrateCamera('./resources_unzipped/AprilBoards.pickle')
print("Camera matrix: ", K)
print("distCoeff: ", dist)

 

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:93: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
Camera matrix:  [[1.94704892e+03 0.00000000e+00 5.71426625e+02]
 [0.00000000e+00 1.82503130e+03 5.36812397e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distCoeff:  [[ 0.09107299  0.1009922  -0.03594065  0.00507835 -0.19710599]]
